I try to analysis question tags.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

In [ ]:
questions = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")

Check Nan

In [ ]:
questions[questions["tags"].isnull()]

The question of 10033 doesn't have tags.

In [ ]:
tags_lst = []
for tags in questions[~questions["tags"].isnull()]["tags"]:
    tags_lst += tags.split()
    
tags_lst = list(map(int, set(tags_lst)))
tags_lst.sort()

len(tags_lst), max(tags_lst)

The tags has 188 numbers, between 0 and 187.

So, create a DataFrame with one-hot encoded tags.

In [ ]:
lst = []
for tags in questions["tags"]:
    ohe = np.zeros(188)
    if str(tags) != "nan":
        for tag in tags.split():
            ohe += np.eye(188)[int(tag)]
    lst.append(ohe)
tags_df = pd.DataFrame(lst, columns=[f"tag_{i}" for i in range(188)]).astype(int)

checking tags of each part 

In [ ]:
for part, df in questions.groupby("part"):
    exist_tags = tags_df.columns[tags_df.loc[df.index].sum() > 0]
    print(f"part {part} has {len(exist_tags)} tags.")

There are some tags that spans multiple parts.

Check multiple tags

In [ ]:
lst = []
for tag in tags_df.columns:
    part_tag = questions.loc[tags_df[tags_df[tag] > 0].index]["part"].unique()
    if len(part_tag) > 1:
        lst.append(",".join(map(str, part_tag)))
        
pd.DataFrame(lst)[0].value_counts()

My insight:
- TOEIC Test part 1 to 4 is listening test, so "1,2,3,4" group's tags may be related listening.
- TOEIC Test part 3 and 4 is listening test which listen long sentence.
- The most frequent "5,6" group may be grammar question because TOEIC test part 5 and 6 are fill-in-the-blank problem.
- "1,2,3,4,6,7" may be a simple problem that applies to any format.


I think that it's enough to pay attention to the tags to understand question types.

## Dimensionality reduction and visualization by PCA

In [ ]:
pca = PCA(n_components=2)

X_2d = pca.fit_transform(tags_df.values)
plt.scatter(X_2d[:,0], X_2d[:,1])

It looks there are some group.

Create this PCA feature DataFrame.

In [ ]:
pca_feat_df = pd.DataFrame(X_2d, columns=["tags_pca_0", "tags_pca_1"])
pca_feat_df["question_id"] = questions["question_id"]
pca_feat_df.head()

In [ ]:
pca_feat_df.to_csv("tags_pca_feat_v1.csv", index=None)

We can use this feature to train model.